## 문장생성 테스트 
* 마르코프 체인
* LSTM/RNN

https://jeongminhee99.tistory.com/81

### 마르코프 체인 

In [2]:
import os
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Twitter
import urllib.request
import os, re, json, random

#네이버 맞춤법 검사 요청에 user-agent 헤더 추가
import requests

### 1. 마르코프 체인 딕셔너리 만들기 
* 마르고프 체인 전용 사전을 만든다. 
* 이 사전은 파이썬 딕셔너리 자료형이며 이전에 설명했던 것처럼 세 단어가 한 세트이다.

In [3]:
# 마르코프 체인 딕셔너리 만들기
def make_dic(words):
    tmp = ["@"]
    dic = {}
    for word in words:
        tmp.append(word)
        if len(tmp) < 3: continue
        if len(tmp) > 3: tmp = tmp[1:]
        set_word3(dic, tmp)
        if word == ".":
            tmp = ["@"]
            continue
    return dic

### 2. 딕셔너리에 데이터 등록하기 
* 딕셔너리에 데이터를 등록한다. 
* 이때 문장의 시작을 나타내는 부분을 "@'로 나타내었다.

In [4]:
# 딕셔너리에 데이터 등록하기 --- (※2)
def set_word3(dic, s3):
    w1, w2, w3 = s3
    if not w1 in dic: dic[w1] = {}
    if not w2 in dic[w1]: dic[w1][w2] = {}
    if not w3 in dic[w1][w2]: dic[w1][w2][w3] = 0
    dic[w1][w2][w3] += 1

### 3. 문장 만들기 
* 여기서는 문장을 만든다. 
* 마르코프 체인의 사전에는 이어서 사용할 수 있는 후보들이 저장되어 있으므로 
* 무작위 후보들을 하나씩 꺼내서 연결하면 나름 그럴듯한 문장이 만들어진다.

In [5]:
# 문장 만들기 --- (※3)
def make_sentence(dic):
    ret = []
    if not "@" in dic: return "no dic" 
    top = dic["@"]
    w1 = word_choice(top)
    w2 = word_choice(top[w1])
    ret.append(w1)
    ret.append(w2)
    while True:
        w3 = word_choice(dic[w1][w2])
        ret.append(w3)
        if w3 == ".": break
        w1, w2 = w2, w3
    ret = "".join(ret)
    # 띄어쓰기
    params = urllib.parse.urlencode({
        "_callback": "",
        "q": ret
    })
    # 네이버 맞춤법 검사기를 사용합니다.
    # data = urllib.request.urlopen("https://m.search.naver.com/p/csearch/ocontent/util/SpellerProxy?" + params)
    data = urllib.request.urlopen("https://m.search.naver.com/p/csearch/ocontent/spellchecker.nhn?" + params)
    data = data.read().decode("utf-8")[1:-2]
    data = json.loads(data)
    data = data["message"]["result"]["html"]
    data = soup = BeautifulSoup(data, "html.parser").getText()
    # 리턴
    return data

### 4. 데이터 학습 및 가공 
* 텍스트를 읽고, 간단하게 가공한다. 
* 형태소 분석을 하고 마르코프 체인을 위한 딕셔너리를 생성한 뒤 JSON 파일로 저장한다.
* 딕셔너리를 기반으로 문장을 생성해서 출력한다.

In [6]:
def word_choice(sel):
    keys = sel.keys()
    return random.choice(list(keys))

# 문장 읽어 들이기 --- (※4)
toji_file = "toji.txt"
dict_file = "markov-toji.json"
if not os.path.exists(dict_file):
    # 토지 텍스트 파일 읽어 들이기
    fp = codecs.open("BEXX0003.txt", "r", encoding="utf-16")
    soup = BeautifulSoup(fp, "html.parser")
    body = soup.select_one("body > text")
    text = body.getText()
    text = text.replace("…", "") # 현재 koNLPy가 …을 구두점으로 잡지 못하는 문제 임시 해결
    # 형태소 분석
   # twitter = Twitter()
    twitter=Twitter()
    malist = twitter.pos(text, norm=True)
    words = []
    for word in malist:
        # 구두점 등은 대상에서 제외(단 마침표는 포함)
        if not word[1] in ["Punctuation"]:
            words.append(word[0])
        if word[0] == ".":
            words.append(word[0])
    # 딕셔너리 생성
    dic = make_dic(words)
    json.dump(dic, open(dict_file,"w", encoding="utf-8"))
else:
    dic = json.load(open(dict_file,"r"))
# 문장 만들기 --- (※6)
for i in range(3):
    s = make_sentence(dic)
    print(s)
    print("---")

FileNotFoundError: [Errno 2] No such file or directory: 'BEXX0003.txt'